## Creating Virtual Environment and Installing Requirments

In [ ]:
python -m venv env

In [ ]:
source env/Scripts/activate

In [ ]:
pip install transformers sentence_transformers

## Getting the similarity between questions

In [ ]:
# Load the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Compute the sentence embeddings
embeddings = model.encode(questions, convert_to_tensor=True)

# Compute the cosine similarity between the sentence embeddings
cosine_scores = util.cos_sim(embeddings[0], embeddings[1])

print('Cosine similarity: {:.4f}'.format(cosine_scores.item()))

## Getting the Candidate lables for each question

In [ ]:
# Getting the candidate label for each question
classifier = pipeline("zero-shot-classification",
                    model="facebook/bart-large-mnli")

question_label1 = classifier(questions[0], candidate_labels)
question_label2 = classifier(questions[1], candidate_labels)

print("Question 1 candidate label: ", question_label1["labels"][0], ", score: ", question_label1["scores"][0])
print("Question 2 candidate label: ", question_label2["labels"][0], ", score: ", question_label2["scores"][0])


## Getting the keywords for each quesitons

In [ ]:
# Getting the keywords for each question
keywordExtractor = pipeline("token-classification",
                    model="yanekyuk/bert-keyword-extractor")

keywords1 = keywordExtractor(questions[0])
keywords2 = keywordExtractor(questions[1])

### Joining the keywords all together

In [ ]:
# Join subwords
keywords_joined1 = []
for keyword in keywords1:
    if keyword['word'].startswith('##'):
        keywords_joined1[-1]['word'] += keyword['word'][2:]
    else:
        keywords_joined1.append(keyword)

keywords_joined2 = []
for keyword in keywords2:
    if keyword['word'].startswith('##'):
        keywords_joined2[-1]['word'] += keyword['word'][2:]
    else:
        keywords_joined2.append(keyword)

# Print only the words
print("Question 1 keywords:")
for keyword in keywords_joined1:
    print("\t", keyword['word'])

print("Question 2 keywords:")
for keyword in keywords_joined2:
    print("\t", keyword['word'])
